## TrpR_IAA

Exemplary design of the trpytophane repressor (PDB: 1ZT9) with the ligand indole-3-acetic acid (IAA).

In [1]:
# This is a comment. It's marked by a leading "#" symbol

# Import OS and system libraries
import sys
import os
import logging

# Append the pocketoptimizer code to your $PYTHONPATH
cwd = os.getcwd()
po_dir = os.path.abspath(os.path.join(cwd, '..'))
project_dir = os.path.join(cwd, 'TrpR_IAA')
if not po_dir in sys.path:
    sys.path.insert(0, po_dir)

# Import Pocketoptimizer
import pocketoptimizer as po

# Remove most warnings, only show Errors
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [2]:
# Initialize a new design pipeline
design = po.DesignPipeline(work_dir=project_dir,         # Path to working directory containing scaffold and ligand subdirectory
                           ph=7,                         # pH used for protein and ligand protonation
                           forcefield='amber_ff14SB',    # forcefield used for all energy computations (Use Amber as it is better tested!)
                           ncpus=8)                      # Number of CPUs for multiprocessing

2023-07-06 11:03:37,791 - pocketoptimizer.ui - INFO - Logging to: /agh/projects/jakob/PycharmProjects/PocketOptimizer2/docs/tutorials/TrpR_IAA/pocketoptimizer.log


### From now on you are inside the directory of your design!

In [3]:
# Prepare ligand
design.parameterize_ligand(input_ligand='ligand/IAA.sdf',  # Input ligand structure file could be .mol2/.sdf
                           addHs=True                      # Whether to add hydrogen atoms to the input structure
                           )

2023-07-06 11:03:54,602 - numexpr.utils - INFO - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-07-06 11:03:54,604 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.
beignet-opencl-icd: no supported GPU found, this is probably the wrong opencl-icd package for this hardware
(If you have multiple ICDs installed and OpenCL works, you can ignore this message)
beignet-opencl-icd: no supported GPU found, this is probably the wrong opencl-icd package for this hardware
(If you have multiple ICDs installed and OpenCL works, you can ignore this message)
2023-07-06 11:04:05,931 - root - WARNING - Warning: importing 'simtk.openmm' is deprecated.  Import 'openmm' instead.
2023-07-06 11:04:12,980 - pocketoptimizer.preparation.structure_building - INFO - Adding hydrogen atoms to the ligand according to pH: 7.
2023-07-06 11:04:15,895 - pocketoptimizer.preparation.structure_building - INFO - Parameterize ligand for GAFF2.
2023-07-06 11:04:16

In [4]:
design.prepare_protein(
    protein_structure='scaffold/1ZT9.pdb',  # Input PDB
    keep_chains=['A', 'B'],  # Specific protein chains to keep
    backbone_restraint=True, # Restrains the backbone during the minimization
    cuda=False,               # Performs minimization on CPU instead of GPU
    discard_mols=[{'chain': 'A', 'resid': '1001'}]     # Special molecules to exclude. Per default everything, but peptides have to be defined manually
    )

2023-07-06 11:04:16,390 - pocketoptimizer.ui - INFO - Start Protein Preparation.
2023-07-06 11:04:16,774 - pocketoptimizer.preparation.structure_building - INFO - Protonate protein according to pH: 7.
2023-07-06 11:04:20,014 - moleculekit.tools.preparation - WARNING - Both chains and segments are defined in Molecule.chain / Molecule.segid, however they are inconsistent. Protein preparation will use the chain information.



---- Molecule chain report ----
Chain A:
    First residue: SER     5  
    Final residue: LEU   105  
Chain B:
    First residue: SER     5  
    Final residue: TRP   901  
---- End of chain report ----



2023-07-06 11:04:22,426 - moleculekit.tools.preparation - INFO - Modified residue HIS    16 A to HID
2023-07-06 11:04:22,427 - moleculekit.tools.preparation - INFO - Modified residue HIS    35 A to HID
2023-07-06 11:04:22,428 - moleculekit.tools.preparation - INFO - Modified residue HIS    16 B to HID
2023-07-06 11:04:22,429 - moleculekit.tools.preparation - INFO - Modified residue HIS    35 B to HID
2023-07-06 11:04:22,430 - moleculekit.tools.preparation - WARNING - Dubious protonation state: the pKa of 2 residues is within 1.0 units of pH 7.0.
2023-07-06 11:04:22,431 - moleculekit.tools.preparation - WARNING - Dubious protonation state:    HIS    16 A (pKa= 6.16)
2023-07-06 11:04:22,432 - moleculekit.tools.preparation - WARNING - Dubious protonation state:    HIS    16 B (pKa= 6.24)
2023-07-06 11:04:26,777 - pocketoptimizer.preparation.structure_building - INFO - Successfully prepared protein structure.
2023-07-06 11:04:26,778 - pocketoptimizer.ui - INFO - Building complex.
2023-07-0

In [5]:
design.prepare_lig_conformers(
    nconfs=50,         # Maximum number of conformers to produce (Sometimes these methods produce lower number of conformations)
    method='genetic',  # Genetic method in OpenBabel, other option is confab
    score='rmsd',      # Filters conformers based on RMSD
    )

2023-07-06 11:04:54,944 - pocketoptimizer.sampling.conformer_generator_obabel - INFO - Starting ligand conformer generation using obabel.
2023-07-06 11:04:54,944 - pocketoptimizer.sampling.conformer_generator_obabel - INFO - Selected Method: genetic.
2023-07-06 11:04:55,642 - pocketoptimizer.sampling.conformer_generator_obabel - INFO - Generated 50 conformers.
2023-07-06 11:04:55,644 - pocketoptimizer.sampling.conformer_generator_obabel - INFO - Conformer sampling was successful.


In [6]:
# Your mutations
design.set_mutations([{'mutations': ['LEU', 'THR'], 'resid': '88', 'chain': 'B'}, 
                      {'mutations': ['ARG'], 'resid': '84', 'chain': 'B'}, 
                      {'mutations': ['LEU','THR'], 'resid': '44', 'chain': 'A'}]
)

2023-07-06 11:04:55,656 - pocketoptimizer.ui - INFO - If design positions are removed or added a new design run should be started.


In [ ]:
# Prepares all defined mutants and glycine scaffolds for side chain rotamer and ligand pose sampling
design.prepare_mutants(sampling_pocket='GLY')

2023-07-06 11:04:56,277 - pocketoptimizer.ui - INFO - Start building mutated protein scaffold variants.
2023-07-06 11:04:56,278 - pocketoptimizer.ui - INFO - Build GLY sampling pockets.
2023-07-06 11:04:56,761 - pocketoptimizer.preparation.structure_building - INFO - Build ligand sampling pocket.
2023-07-06 11:04:57,224 - htmd.builder.amber - INFO - Detecting disulfide bonds.
2023-07-06 11:04:57,470 - pocketoptimizer.preparation.structure_building - INFO - Starting the build.
2023-07-06 11:04:57,813 - pocketoptimizer.preparation.structure_building - INFO - Finished building.
2023-07-06 11:04:59,280 - pocketoptimizer.preparation.structure_building - INFO - Build mutation: A_44_LEU.
2023-07-06 11:04:59,781 - htmd.builder.amber - INFO - Detecting disulfide bonds.
2023-07-06 11:04:59,954 - pocketoptimizer.preparation.structure_building - INFO - Starting the build.
2023-07-06 11:05:00,304 - pocketoptimizer.preparation.structure_building - INFO - Finished building.
2023-07-06 11:05:01,750 - 

In [ ]:
# Sampling of side chain rotamers
design.sample_sidechain_rotamers(
    vdw_filter_thresh=100,         # Energy threshold of 100 kcal/mol for filtering rotamers
    library='dunbrack',            # Use dunbrack rotamer library (Should be used!)
    dunbrack_filter_thresh=0.001,  # Probability threshold for filtering rotamers (0.1%)
    accurate=False,                # Increases the number of rotamers sampled when using dunbrack (Be careful about the computation time!)
    include_native=True            # Include the native rotamers from the minimized structure
)

In [ ]:
# Sampling of ligand poses
# Defines a grid in which the ligand is translated and rotated along.
#                       Range, Steps
sample_grid = {'trans': [1, 0.5],  # Angstrom
               'rot': [20, 20]}    # Degree
design.sample_lig_poses(
    method='grid',         #  Uses the grid method. Other option is random
    grid=sample_grid,      #  Defined grid for sampling    
    vdw_filter_thresh=100, #  Energy threshold of 100 kcal/mol for filtering ligand poses
    max_poses=10000        #  Maximum number of poses
)

In [ ]:
design.calculate_energies(
    scoring='vina',           #  Method to score protein-ligand interaction
)

In [ ]:
# Compute the lowest energy structures using linear programming
design.design(
    num_solutions=10,           #  Number of solutions to compute
    ligand_scaling=10,          #  Scaling factor for binding-related energies (You need to adapt this to approximate the packing and binding energies)
)

In [3]:
# Do not run this unless you want to fully clean your working directory from all created files and folders!
design.clean(scaffold=True, ligand=True)

2023-07-06 11:03:26,271 - pocketoptimizer.ui - INFO - All scaffold files were deleted.
2023-07-06 11:03:26,310 - pocketoptimizer.ui - INFO - All ligand files were deleted.
2023-07-06 11:03:26,332 - pocketoptimizer.ui - INFO - Logfile was deleted.
